# Todo list: 
<li> Find a way to "trigger" messege to kafka (May not need to store in hdfs anymore-> contr)</li>
<li> Push to github </li>

In [ ]:
!wget http://central.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.11/2.4.1/spark-sql-kafka-0-10_2.11-2.4.1.jar

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.types import StructType
import json
emailSchema = StructType() \
        .add("subject", StringType()) \
        .add("body", StringType())

# Group the data by window and subject and compute the count of each group

<li>The engine will maintain state and allow late data to be update the state until (max event time seen by the engine - late threshold > 1 Month). In other words, late data within the threshold will be aggregated, but data later than the threshold will be dropped.</li> 
<li>Count subject within 1 day windows, updating every 40 minutes</li>

In [4]:

# Subscribe to 1 topic

# .option("kafka.bootstrap.servers", "188.185.79.229:9092")\
# .option("startingOffsets", "earliest") \

data = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "188.185.79.229:9092")\
.option("subscribe","mock_email_alert")\
.load()\
.select(
    from_json(col("value").cast("string"), emailSchema).getField("subject").alias("subject"),
    col("timestamp").alias("timestamp"))\

data.printSchema()

root
 |-- subject: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [22]:
filter_data=data.withWatermark("timestamp", "1 minute")\
.groupBy(window('timestamp', "5 minute", "5 seconds"),"subject")\
.agg(count("subject"))\
# .select("*").where("count(subject) > 10")

filter_data.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- subject: string (nullable = true)
 |-- count(subject): long (nullable = false)



In [32]:
streamout_data=data\
.select( \
    to_json(struct("subject")).alias("key"),
    col("subject").cast("string").alias("value")) \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("topic", "filter_alert") \
  .option("checkpointLocation", "/user/ysunthor/checkpoint") \
  .outputMode("append") \

In [23]:
data_flow = data.writeStream.queryName("email").outputMode("Append").format("memory").start()
filter_flow = filter_data.writeStream.queryName("filter_email").outputMode("Append").format("memory").start()
# streamout_flow = streamout_data.start()

In [127]:
data.isStreaming

True

In [34]:
spark.streams.active

[]

In [25]:
data_flow.lastProgress
# filter_flow.lastProgress
# streamout_flow.lastProgress

In [33]:
data_flow.stop()
filter_flow.stop()
# streamout_flow.stop()

In [30]:
alerts = spark.sql("select * from email")
alerts.show()

+--------------+--------------------+
|       subject|           timestamp|
+--------------+--------------------+
|      CRON job|2019-06-11 16:26:...|
|      CRON job|2019-06-11 16:26:...|
|PopDB problems|2019-06-11 16:27:...|
|PopDB problems|2019-06-11 16:27:...|
|       WARNING|2019-06-11 16:27:...|
|       WARNING|2019-06-11 16:27:...|
|PopDB problems|2019-06-11 16:27:...|
|PopDB problems|2019-06-11 16:27:...|
|PopDB problems|2019-06-11 16:27:...|
|       WARNING|2019-06-11 16:27:...|
|       WARNING|2019-06-11 16:27:...|
|PopDB problems|2019-06-11 16:27:...|
|      CRON job|2019-06-11 16:27:...|
|      CRON job|2019-06-11 16:27:...|
|       WARNING|2019-06-11 16:28:...|
|       WARNING|2019-06-11 16:28:...|
|       WARNING|2019-06-11 16:28:...|
|       WARNING|2019-06-11 16:28:...|
|PopDB problems|2019-06-11 16:28:...|
|      CRON job|2019-06-11 16:28:...|
+--------------+--------------------+
only showing top 20 rows



In [31]:
alerts = spark.sql("select * from filter_email")
alerts.show()

+--------------------+--------------+--------------+
|              window|       subject|count(subject)|
+--------------------+--------------+--------------+
|[2019-06-11 16:21...|      CRON job|             1|
|[2019-06-11 16:22...|      CRON job|             2|
|[2019-06-11 16:22...|       WARNING|             2|
|[2019-06-11 16:23...|       WARNING|             4|
|[2019-06-11 16:22...|       WARNING|             3|
|[2019-06-11 16:22...|      CRON job|             2|
|[2019-06-11 16:22...|      CRON job|             2|
|[2019-06-11 16:22...|       WARNING|             4|
|[2019-06-11 16:22...|PopDB problems|             5|
|[2019-06-11 16:22...|      CRON job|             2|
|[2019-06-11 16:22...|      CRON job|             2|
|[2019-06-11 16:22...|PopDB problems|             5|
|[2019-06-11 16:22...|PopDB problems|             6|
|[2019-06-11 16:23...|PopDB problems|             6|
|[2019-06-11 16:22...|PopDB problems|             1|
|[2019-06-11 16:23...|PopDB problems|         